In [36]:
pip install dtale

     |████████████████████████████████| 7.7 MB 11.9 MB/s eta 0:00:01    |██████████████████              | 4.3 MB 1.4 MB/s eta 0:00:03
     |████████████████████████████████| 185 kB 39.8 MB/s eta 0:00:01
     |████████████████████████████████| 720 kB 34.7 MB/s eta 0:00:01     |█████████████████████▉          | 491 kB 34.7 MB/s eta 0:00:01
     |████████████████████████████████| 45 kB 3.7 MB/s  eta 0:00:01
     |████████████████████████████████| 62 kB 2.2 MB/s  eta 0:00:01
     |████████████████████████████████| 642 kB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 72 kB 3.1 MB/s  eta 0:00:01
     |████████████████████████████████| 251 kB 2.4 MB/s eta 0:00:01
     |████████████████████████████████| 421 kB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 13.1 MB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 3.4 MB 522 kB/s eta 0:00:01     |███████▍                        | 788 kB 

  Created wheel for dash-table: filename=dash_table-4.10.1-py3-none-any.whl size=1824996 sha256=2fb431cd6b53e3ece7f06bbebefd91e6c9cdae645fc6d1a67d38a61a6a0e18a8
  Stored in directory: /Users/m1nsun9/Library/Caches/pip/wheels/0c/5b/84/58c763a6dc2cc332fe404901748ad89d25d9dacd48778ee241
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11430 sha256=6ea360f72af9adea0e7b459c5498902ed8e8c015ccc92b7e60ebeda63506a370
  Stored in directory: /Users/m1nsun9/Library/Caches/pip/wheels/c4/a7/48/0a434133f6d56e878ca511c0e6c38326907c0792f67b476e56
Successfully built Flask-Compress dash-colorscales dash-daq dash dash-renderer dash-core-components dash-html-components dash-table retrying
Note: you may need to restart the kernel to use updated packages.


In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import dtale

In [30]:
base = '../output/'

historical_prices_df = pd.read_csv(base + 'nasdaq_historical_prices.csv')
historical_prices_df = historical_prices_df.drop(columns='Unnamed: 0')
historical_prices_df

,Company,Q1 Date,Q1 Price,Q2 Date,Q2 Price,Q3 Date,Q3 Price
0,DOCU,2020-01-31,78.5100,2020-04-30,104.7500,2020-07-31,216.830000
1,ZM,2020-01-31,76.3000,2020-04-30,135.1700,2020-07-31,253.910000
2,DXCM,2019-12-31,218.7400,2020-03-31,269.2700,2020-06-30,405.400000
3,SPLK,2020-01-31,155.2600,2020-04-30,140.3600,2020-07-31,209.820000
4,SGEN,2019-12-31,114.2600,2020-03-31,115.3800,2020-06-30,169.920000
...,...,...,...,...,...,...,...
98,NVDA,2020-01-26,NaN,2020-04-26,NaN,2020-07-26,NaN
99,PAYX,2020-02-29,NaN,2020-05-31,NaN,2020-08-31,76.470001
100,KLAC,2019-12-31,175.6855,2020-03-31,142.4356,2020-06-30,193.646300
101,CSCO,2020-01-25,NaN,2020-04-25,NaN,2020-07-25,NaN


In [31]:
financial_ratios_df = pd.read_csv(base + 'nasdaq_financial_ratios.csv')
financial_ratios_df = financial_ratios_df.drop(columns='Unnamed: 0')
financial_ratios_df

,Company,Q1 P/E,Q2 P/E,Q3 P/E,Q1 ROE,Q2 ROE,Q3 ROE,Q1 Date,Q2 Date,Q3 Date
0,DOCU,-299.597287,-400.948571,-620.874031,-0.086774,-0.091441,-0.128634,2020-01-31,2020-04-30,2020-07-31
1,ZM,1191.100353,1473.689789,405.682498,0.018393,0.030029,0.155144,2020-01-31,2020-04-30,2020-07-31
2,DXCM,218.504035,1273.281759,849.326134,0.105031,0.021295,0.034198,2019-12-31,2020-03-31,2020-06-30
3,SPLK,-1064.434087,-72.359266,-127.625338,-0.011367,-0.168823,-0.156529,2020-01-31,2020-04-30,2020-07-31
4,SGEN,759.556952,-118.084957,-1390.521355,0.013767,-0.095379,-0.011681,2019-12-31,2020-03-31,2020-06-30
...,...,...,...,...,...,...,...,...,...,...
98,NVDA,156.637749,201.356865,419.415270,0.077925,0.070005,0.044703,2020-01-26,2020-04-26,2020-07-26
99,PAYX,77.441170,116.583405,130.569997,0.128451,0.079349,0.076153,2020-02-29,2020-05-31,2020-08-31
100,KLAC,73.227875,285.357774,73.578936,0.142543,0.034000,0.154292,2019-12-31,2020-03-31,2020-06-30
101,CSCO,71.068582,64.528627,74.669499,0.080995,0.077686,0.069515,2020-01-25,2020-04-25,2020-07-25


In [45]:
financial_growth_df = pd.read_csv(base + 'nasdaq_financial_growth.csv')
dtale.show(financial_growth_df)

In [46]:
nasdaq_df = historical_prices_df.merge(financial_ratios_df, how='outer', on=['Company', 'Q1 Date', 'Q2 Date', 'Q3 Date'])

dtale.show(nasdaq_df)

In [49]:
clean_nasdaq_df = nasdaq_df.merge(financial_growth_df, how='outer', on=['Company', 'Q1 Date', 'Q2 Date', 'Q3 Date'])

In [55]:
# remove [SIRI BIDU INTC TCOM] from df

clean_nasdaq_df = clean_nasdaq_df.loc[(clean_nasdaq_df['Company'] != 'BIDU') & (clean_nasdaq_df['Company'] != 'SIRI')
                                     & (clean_nasdaq_df['Company'] != 'INTC') & (clean_nasdaq_df['Company'] != 'TCOM')]

clean_nasdaq_df = clean_nasdaq_df.reset_index(drop=True)

# change price to that 12/27/19
clean_nasdaq_df.loc['AAPL', 'Q1 Price'] = 72.449997

# change price to that 3/27/20
clean_nasdaq_df.loc['AAPL', 'Q2 Price'] = 61.935001

# change price to that of 6/26/20
clean_nasdaq_df.loc['AAPL', 'Q3 Price'] = 88.407501

dtale.show(clean_nasdaq_df, ignore_duplicate=True)

In [56]:
clean_nasdaq_df = clean_nasdaq_df.dropna()
clean_nasdaq_df = clean_nasdaq_df.reset_index(drop=True)

In [57]:
dtale.show(clean_nasdaq_df, ignore_duplicate=True)

### 

In [58]:
big_tech_df = clean_nasdaq_df.loc[(clean_nasdaq_df['Company'] == 'FB') | (clean_nasdaq_df['Company'] == 'AMZN') |  
                                  (clean_nasdaq_df['Company'] == 'AAPL') | 
                                  (clean_nasdaq_df['Company'] == 'MSFT') | 
                                  (clean_nasdaq_df['Company'] ==  'NFLX') | 
                                  (clean_nasdaq_df['Company'] == 'GOOG')]

big_tech_df = big_tech_df.reset_index(drop=True)
dtale.show(big_tech_df, ignore_duplicate=True)